In [1]:
# Cell 2: PyTorchESN Class Definition
import torch
import torch.nn as nn
import numpy as np

class PyTorchESN(nn.Module):
    def __init__(self,
                 input_dim,
                 reservoir_dim,
                 output_dim,
                 spectral_radius=0.9,
                 W_res_density=0.1,
                 input_scaling=1.0,
                 bias_scaling=0.2,
                 leak_rate=1.0,
                 activation_func=torch.tanh,
                 Win_data=None,
                 W_res_data=None,
                 bias_res_data=None,
                 W_out_data=None,
                 bias_out_data=None,
                 readout_uses_input=False):

        super(PyTorchESN, self).__init__()

        # ---- Store Hyperparameters and Configuration ----
        self.input_dim = input_dim
        self.reservoir_dim = reservoir_dim
        self.output_dim = output_dim
        self.spectral_radius = spectral_radius
        self.W_res_density = W_res_density
        self.input_scaling = input_scaling
        self.bias_scaling = bias_scaling
        self.leak_rate = leak_rate
        self.activation_func = activation_func
        self.readout_uses_input = readout_uses_input

        # This method handles all weight initializations
        self.load_weights_from_data(Win_data, W_res_data, bias_res_data, W_out_data, bias_out_data)

        self.current_reservoir_state = None
        self.is_initial_state = True

    def _initialize_W_res(self):
        W_res_ = torch.randn(self.reservoir_dim, self.reservoir_dim)
        num_zero_elements = int((1 - self.W_res_density) * self.reservoir_dim**2)
        if 0 < num_zero_elements < self.reservoir_dim**2:
            zero_indices = torch.randperm(self.reservoir_dim**2)[:num_zero_elements]
            W_res_.view(-1)[zero_indices] = 0
        
        if self.reservoir_dim > 0:
            try:
                eigenvalues = torch.linalg.eigvals(W_res_)
                current_spectral_radius = torch.max(torch.abs(eigenvalues))
                if current_spectral_radius > 1e-9:
                    W_res_scaled = W_res_ * (self.spectral_radius / current_spectral_radius)
                else:
                    W_res_scaled = W_res_
            except Exception:
                W_res_scaled = W_res_
        else:
            W_res_scaled = W_res_
        return W_res_scaled

    def _update_reservoir_state(self, current_input, previous_reservoir_state):
        """
        Computes one step of the reservoir state update using the correct matrix multiplication order.
        r(t) = (1-lr)*r(t-1) + lr*activation( W_in @ x(t) + W_res @ r(t-1) + bias_res )
        """
        input_contrib = torch.matmul(self.Win, current_input.T).T
        reservoir_contrib = torch.matmul(self.W_res, previous_reservoir_state.T).T
        pre_activation = input_contrib + reservoir_contrib + self.bias_res
        activated_state = self.activation_func(pre_activation)
        new_reservoir_state = (1 - self.leak_rate) * previous_reservoir_state + self.leak_rate * activated_state
        return new_reservoir_state
    
    def forward(self, input_sequence, initial_reservoir_state=None):
        batch_size, sequence_length, _ = input_sequence.shape
        device = input_sequence.device
        
        if initial_reservoir_state is None:
            current_reservoir_state = torch.zeros(batch_size, self.reservoir_dim, device=device)
        else:
            current_reservoir_state = initial_reservoir_state.to(device)

        # Since seq_length is 1 for this app, this loop runs only once.
        for t in range(sequence_length):
            current_input_t = input_sequence[:, t, :]
            current_reservoir_state = self._update_reservoir_state(current_input_t, current_reservoir_state)
            
        readout_input = current_reservoir_state
        if self.readout_uses_input:
            readout_input = torch.cat((current_reservoir_state, current_input_t), dim=1)
        
        final_output = self.W_out_layer(readout_input)
        
        return final_output, current_reservoir_state

    def reset_state(self):
        self.current_reservoir_state = None
        self.is_initial_state = True
        
    def load_weights_from_data(self, Win_data, W_res_data, bias_res_data, W_out_data, bias_out_data):
        if Win_data is not None:
            self.Win = nn.Parameter(torch.tensor(Win_data, dtype=torch.float32), requires_grad=False)
        else:
            Win_ = torch.rand(self.reservoir_dim, self.input_dim) * 2 - 1
            self.Win = nn.Parameter(Win_ * self.input_scaling, requires_grad=False)
            
        if bias_res_data is not None:
            self.bias_res = nn.Parameter(torch.tensor(bias_res_data.flatten(), dtype=torch.float32), requires_grad=False)
        else:
            bias_res_ = torch.rand(self.reservoir_dim) * 2 - 1
            self.bias_res = nn.Parameter(bias_res_ * self.bias_scaling, requires_grad=False)
            
        if W_res_data is not None:
            self.W_res = nn.Parameter(torch.tensor(W_res_data, dtype=torch.float32), requires_grad=False)
        else:
            self.W_res = nn.Parameter(self._initialize_W_res(), requires_grad=False)

        readout_input_dim = self.reservoir_dim + (self.input_dim if self.readout_uses_input else 0)
        self.W_out_layer = nn.Linear(readout_input_dim, self.output_dim, bias=True)

        if W_out_data is not None:
            W_out_numpy = np.asarray(W_out_data)
            expected_shape = (self.output_dim, readout_input_dim)
            if W_out_numpy.shape == expected_shape:
                W_out_to_load = W_out_numpy
            elif W_out_numpy.shape == (expected_shape[1], expected_shape[0]):
                W_out_to_load = W_out_numpy.T
            else:
                raise ValueError(f"Shape mismatch for W_out_data. Expected {expected_shape} or transpose, got {W_out_numpy.shape}")
            self.W_out_layer.weight = nn.Parameter(torch.tensor(W_out_to_load, dtype=torch.float32), requires_grad=False)
        else:
            nn.init.zeros_(self.W_out_layer.weight)

        if bias_out_data is not None:
            bias_out_numpy = np.asarray(bias_out_data).flatten()
            if bias_out_numpy.shape[0] != self.output_dim:
                raise ValueError(f"Shape mismatch for bias_out_data. Expected ({self.output_dim},), got {bias_out_numpy.shape}")
            self.W_out_layer.bias = nn.Parameter(torch.tensor(bias_out_numpy, dtype=torch.float32), requires_grad=False)
        else:
            if self.W_out_layer.bias is not None:
                nn.init.zeros_(self.W_out_layer.bias)

In [2]:
# Cell 3: Train ReservoirPy and Instantiate PyTorch ESN
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from aeon.datasets import load_classification
from reservoirpy.nodes import Reservoir, Ridge

# ========== 1. Load and Prepare Data ==========
print("--- Loading and Preparing Data ---")
X_train_orig, y_train_orig = load_classification('ECG5000', split='train')
X_test_orig, y_test_orig = load_classification('ECG5000', split='test')

# Squeeze the middle dimension (n_dims=1)
X_train = X_train_orig.squeeze()
X_test = X_test_orig.squeeze()

# Flatten each time-series sample into a single feature vector
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

# Encode labels and convert to one-hot format
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train_orig)
y_test = label_encoder.transform(y_test_orig)
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
y_test_labels = np.argmax(y_test_cat, axis=1)

print(f"Flattened training data shape: {X_train_reshaped.shape}")
print(f"One-hot training labels shape: {y_train_cat.shape}")

# ========== 2. Define and Train ReservoirPy ESN Model ==========
print("\n--- Training ReservoirPy ESN Model ---")
INPUT_DIM = X_train_reshaped.shape[1]
RESERVOIR_DIM = 200
OUTPUT_DIM = y_train_cat.shape[1]

reservoir = Reservoir(units=RESERVOIR_DIM, input_dim=INPUT_DIM, sr=0.4, input_connectivity=0.8,
                      rc_connectivity=0.1, input_scaling=0.3, lr=1, seed=43)
readout = Ridge(output_dim=OUTPUT_DIM, ridge=1e-1)
esn_model = reservoir >> readout

esn_model.fit(X_train_reshaped, y_train_cat)
print("ReservoirPy ESN training complete.")

# ========== 3. Calculate ReservoirPy Model Accuracy ==========
y_pred_rp = esn_model.run(X_test_reshaped)
y_pred_labels_rp = np.argmax(y_pred_rp, axis=1)
acc_rp = np.mean(y_pred_labels_rp == y_test_labels)
print(f"ReservoirPy Original Accuracy: {acc_rp*100:.2f}%")

# ========== 4. Get Effective Weights from ReservoirPy ==========
print("\n--- Extracting Effective Weights from ReservoirPy Model ---")
Win_eff = (reservoir.Win.toarray())
W_res_eff = reservoir.W.toarray()
bias_res_eff = (reservoir.bias.toarray().flatten())
W_out_eff = readout.Wout
bias_out_eff = readout.bias

# ========== 5. Instantiate the PyTorchESN Model ==========
print("\n--- Instantiating PyTorchESN with Loaded Weights ---")
pytorch_esn_opt1 = PyTorchESN(
    input_dim=INPUT_DIM,
    reservoir_dim=RESERVOIR_DIM,
    output_dim=OUTPUT_DIM,
    Win_data=Win_eff,
    W_res_data=W_res_eff,
    bias_res_data=bias_res_eff,
    W_out_data=W_out_eff,
    bias_out_data=bias_out_eff,
    leak_rate=reservoir.lr,
    spectral_radius=reservoir.sr,
    readout_uses_input=False
)
pytorch_esn_opt1.eval()
print("PyTorchESN instantiated and ready for verification.")

2025-07-20 11:05:28.576548: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-20 11:05:28.583708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753002328.592494  641111 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753002328.595033  641111 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753002328.601656  641111 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

--- Loading and Preparing Data ---
Flattened training data shape: (500, 140)
One-hot training labels shape: (500, 5)

--- Training ReservoirPy ESN Model ---


Running Model-0: 500it [00:00, 15412.53it/s]          
Running Model-0: 100%|██████████| 1/1 [00:00<00:00, 28.93it/s]

Fitting node Ridge-0...


ReservoirPy ESN training complete.


Running Model-0: 4500it [00:00, 16273.18it/s]         

ReservoirPy Original Accuracy: 93.04%

--- Extracting Effective Weights from ReservoirPy Model ---

--- Instantiating PyTorchESN with Loaded Weights ---
PyTorchESN instantiated and ready for verification.


In [3]:
# Cell 4: Verify PyTorch Model
# Reshape the flattened test data to (n_samples, 1, n_features) for the PyTorch model
import numpy

X_test_torch = torch.tensor(X_test_reshaped, dtype=torch.float32).unsqueeze(1)

print(f"PyTorch test input shape: {X_test_torch.shape}")

# Get PyTorch ESN Predictions
pytorch_esn_opt1.eval()
pytorch_esn_opt1.reset_state()
with torch.no_grad():
    y_pred_torch, _ = pytorch_esn_opt1(X_test_torch)

y_pred_labels_torch = np.argmax(y_pred_torch.numpy(), axis=1)
acc_torch = np.mean(y_pred_labels_torch == y_test_labels)

print("\n--- Verification ---")
print(f"ReservoirPy Original Accuracy:      {acc_rp*100:.2f}%")
print(f"PyTorch Re-implementation Accuracy: {acc_torch*100:.2f}%")

if np.isclose(acc_rp, acc_torch, atol=1e-2):
    print("\nSUCCESS: Accuracies are very close. Proceeding to ONNX export.")
else:
    print("\nWARNING: Accuracies differ significantly. Please review the implementation.")

PyTorch test input shape: torch.Size([4500, 1, 140])

--- Verification ---
ReservoirPy Original Accuracy:      93.04%
PyTorch Re-implementation Accuracy: 92.80%

SUCCESS: Accuracies are very close. Proceeding to ONNX export.


In [4]:
General_Batch_Size = 2048


In [5]:
# Cell 5: Export to ONNX
ONNX_MODEL_PATH = "esn_fp32.onnx"
DUMMY_BATCH_SIZE = General_Batch_Size
DUMMY_SEQUENCE_LENGTH = 1 # Fixed sequence length of 1

pytorch_esn_opt1.eval()
pytorch_esn_opt1.to('cpu')
pytorch_esn_opt1.reset_state()

dummy_input_sequence = torch.randn(
    DUMMY_BATCH_SIZE, DUMMY_SEQUENCE_LENGTH, INPUT_DIM, device='cpu', dtype=torch.float32
)

print(f"\n--- Exporting to ONNX: {ONNX_MODEL_PATH} ---")
torch.onnx.export(
    pytorch_esn_opt1,
    (dummy_input_sequence,),
    ONNX_MODEL_PATH,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input_sequence'],
    output_names=['final_output', 'final_reservoir_state'],
    dynamic_axes={
        'input_sequence': {0: 'batch_size'},
        'final_output': {0: 'batch_size'},
        'final_reservoir_state': {0: 'batch_size'}
    }
)
print("ONNX export successful.")
print(f"FP32 ONNX model saved to: {ONNX_MODEL_PATH}")


--- Exporting to ONNX: esn_fp32.onnx ---
ONNX export successful.
FP32 ONNX model saved to: esn_fp32.onnx


In [6]:
# Step 4: Post-Training Quantization to create INT8 ONNX model

import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

ONNX_QUANT_PATH = "esn_uint8.onnx"

print("\nPerforming dynamic quantization for ONNX model...")

quantize_dynamic(
    model_input=ONNX_MODEL_PATH,
    model_output=ONNX_QUANT_PATH,
    weight_type=QuantType.QUInt8 # Use UINT8 for ARM, as it's often well-supported
)

print(f"Quantized INT8 ONNX model saved to: {ONNX_QUANT_PATH}")


Performing dynamic quantization for ONNX model...
Ignore MatMul due to non constant B: /[/MatMul]
Ignore MatMul due to non constant B: /[/MatMul_1]
Quantized INT8 ONNX model saved to: esn_uint8.onnx


In [7]:
X_test_reshaped 

array([[ 3.6908442 ,  0.71141435, -2.1140915 , ...,  1.0359968 ,
         1.4922866 , -1.9050734 ],
       [-1.3481323 , -3.9960376 , -4.2267496 , ...,  0.26753217,
         1.0711484 , -1.164009  ],
       [ 1.0242946 , -0.59031419, -1.9169491 , ...,  1.8163009 ,
         1.4739633 ,  1.3897666 ],
       ...,
       [-1.3517791 , -2.2090058 , -2.5202247 , ..., -2.2600228 ,
        -1.577823  , -0.68453092],
       [-1.1244318 , -1.9050388 , -2.1927069 , ..., -0.4433066 ,
        -0.55976901,  0.10856792],
       [ 0.72881283,  0.19259731, -0.73388442, ...,  1.5389752 ,
         1.713781  ,  1.3093816 ]])

In [8]:
y_test_labels

array([0, 0, 0, ..., 1, 1, 1])

In [9]:
# --- Save the necessary arrays to .npy files ---
X_TEST_FILE = 'X_test_reshaped.npy'
Y_TEST_FILE = 'y_test_labels.npy'

print(f"Saving test data to files...")
np.save(X_TEST_FILE, X_test_reshaped)
np.save(Y_TEST_FILE, y_test_labels)

print(f"Successfully saved test data to '{X_TEST_FILE}' and '{Y_TEST_FILE}'.")
print("You can now transfer these files to your Zynq board.")

Saving test data to files...
Successfully saved test data to 'X_test_reshaped.npy' and 'y_test_labels.npy'.
You can now transfer these files to your Zynq board.


In [10]:
# # Step 6-7: Final benchmarking script to run ON THE ZYNQ

# import onnxruntime as ort
# import numpy as np
# import timeit
# from sklearn.metrics import accuracy_score

# # Add this class to your Python script on the Zynq

# import threading
# import time
# import os
# import numpy as np

# class ZynqPowerMonitor:
#     """
#     A thread-based power monitor for Zynq UltraScale+ MPSoCs by reading sysfs files.
#     """
#     def __init__(self, hwmon_path=None, interval_sec=0.1):
#         """
#         Args:
#             hwmon_path (str, optional): The full path to the hwmon directory. 
#                                         If None, it will try to find it automatically.
#             interval_sec (float): The polling interval in seconds.
#         """
#         self.interval = interval_sec
#         self.power_readings_watts = []
        
#         self._stop_event = threading.Event()
#         self._monitor_thread = None

#         if hwmon_path:
#             self.power_file_path = os.path.join(hwmon_path, 'power1_input')
#         else:
#             # Attempt to find the power file automatically
#             self.power_file_path = self._find_power_file()

#         if not os.path.exists(self.power_file_path):
#             raise FileNotFoundError(f"Power monitor file not found at '{self.power_file_path}'. "
#                                     "Please find the correct path using 'find /sys/bus/i2c/devices/ -name \"power1_input\"'")

#     def _find_power_file(self):
#         """Tries to locate the 'power1_input' file automatically."""
#         for root, dirs, files in os.walk('/sys/bus/i2c/devices/'):
#             if 'power1_input' in files:
#                 # This logic could be improved to pick a specific sensor if there are multiple.
#                 # For most boards, the first one found is for the main PS rails.
#                 return os.path.join(root, 'power1_input')
#         return None

#     def _get_power_reading_watts(self):
#         """Reads the sysfs file and converts from microwatts to Watts."""
#         try:
#             with open(self.power_file_path, 'r') as f:
#                 power_microwatts = int(f.read())
#             # Convert from microwatts to Watts
#             return power_microwatts / 1_000_000.0
#         except Exception as e:
#             print(f"Warning: Could not read power file. Error: {e}")
#             return None

#     def _monitor_power_loop(self):
#         """The main loop for the monitoring thread."""
#         while not self._stop_event.is_set():
#             power = self._get_power_reading_watts()
#             if power is not None:
#                 self.power_readings_watts.append(power)
#             time.sleep(self.interval)

#     def start(self):
#         """Starts the power monitoring background thread."""
#         print("Starting Zynq power monitor...")
#         self.power_readings_watts = []
#         self._stop_event.clear()
#         self.start_time = time.time()
#         self._monitor_thread = threading.Thread(target=self._monitor_power_loop)
#         self._monitor_thread.start()

#     def stop(self):
#         """Stops the monitoring thread and returns the calculated metrics."""
#         self._stop_event.set()
#         if self._monitor_thread is not None:
#             self._monitor_thread.join()
#         self.end_time = time.time()
#         print("Power monitor stopped.")
        
#         duration_sec = self.end_time - self.start_time
#         avg_power = np.mean(self.power_readings_watts) if self.power_readings_watts else 0.0
#         energy_j = avg_power * duration_sec
#         latency_ms = duration_sec * 1000
        
#         return latency_ms, avg_power, energy_j

# # --- Helper function for ONNX Runtime inference ---
# def run_ort_inference(model_path, test_data, execution_provider):
#     # Set session options and specify the execution provider
#     sess_options = ort.SessionOptions()
#     ort_session = ort.Session(model_path, sess_options, providers=[execution_provider])
    
#     input_name = ort_session.get_inputs()[0].name
    
#     # Run inference for all test data
#     predictions = ort_session.run(None, {input_name: test_data})[0]
#     return predictions

# # --- Main Execution on Zynq ---
# if __name__ == '__main__':
#     # Load your test data on the Zynq
#     X_test_reshaped = np.load('X_test_reshaped.npy').astype(np.float32)
#     # Reshape to (batch_size, 1, features)
#     test_input_data_np = X_test_reshaped.reshape(-1, 1, X_test_reshaped.shape[1])
#     y_test_labels = np.load('y_test_labels.npy')
    
#     zynq_monitor = ZynqPowerMonitor()

#     models_to_benchmark = {
#         "ONNX FP32": "esn_fp32.onnx",
#         "ONNX INT8": "esn_uint8.onnx"
#     }
    
#     print("\n--- Zynq Performance & Accuracy Report ---")
#     print("="*100)
#     header = f"{'Model':<15} | {'Accuracy (%)':<15} | {'Latency (s)':<15} | {'Throughput (inf/s)':<20} | {'Avg Power (W)':<15} | {'Energy (J)':<12}"
#     print(header)
#     print("-" * 100)

#     for name, path in models_to_benchmark.items():
#         print(f"Benchmarking {name}...")
        
#         # Use 'CPUExecutionProvider', which is optimized for ARM NEON
#         exec_provider = 'CPUExecutionProvider'

#         # Time the full inference run
#         def workload():
#             return run_ort_inference(path, test_input_data_np, exec_provider)
        
#         # Warm-up run
#         _ = workload()

#         # Measure power and energy for a single, full-batch run
#         zynq_monitor.start()
#         predictions = workload()
#         latency, avg_power, energy = zynq_monitor.stop()

#         latency_sec = latency / 1000.0
#         throughput = len(X_test_reshaped) / latency_sec
        
#         # Calculate accuracy
#         pred_labels = np.argmax(predictions, axis=1)
#         acc = accuracy_score(y_test_labels, pred_labels)
        
#         # Print results
#         print(f"{name:<15} | {acc*100:<15.2f} | {latency_sec:<15.4f} | {throughput:<20.2f} | {avg_power:<15.4f} | {energy:<12.4f}")

In [11]:
# pip install onnxruntime

In [12]:
# import onnxruntime as ort
# onnxruntime__version__ = ort.__version__
# print (onnxruntime__version__)

In [13]:
# ort.InferenceSession()

In [14]:
# pip install --upgrade --force-reinstall --ignore-installed -r reqs.txt